# 车牌识别大作业

李沿橙 1900012766

In [ ]:
# import collection
import cv2
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import matplotlib.patches as patches

import visdom
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from collections import OrderedDict
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
torch.manual_seed(2)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## 一、数据准备

In [ ]:
# 获取一个目录下所有文件的文件名（包含 path），返回一个列表，按字典序返回
def get_all_files(path):
    return ["\\".join([path, x]) for x in os.listdir(path)]

In [ ]:
root_path_train = r'.\Plate_dataset\AC\train\xml' # 训练集目录
root_path_test = r'.\Plat_dataset\AC\test\xml'

In [76]:
# 根据 path 指定的 xml 文件，返回一个二元组，第一个元素是裁剪后的车牌图片，第二个是车牌号码
def xml_to_tuple(path):
    xml_loader = ET.ElementTree(file = path)
    second = xml_loader.find('object').find('platetext').text
    
    filename = xml_loader.find('filename').text
    filename = '\\'.join(path.split('\\')[:-2]) + '\\jpeg\\' + filename # 图片的完整路径

    img = cv2.imread(filename)
    bbox = xml_loader.find('object').find('bndbox')
    xmin = bbox.find('xmin').text
    ymin = bbox.find('ymin').text
    xmax  = bbox.find('xmax').text
    ymax = bbox.find('ymax').text
    bbox = [xmin, ymin, xmax, ymax]
    bbox = [int(b) for b in bbox] # left, top, right, bottom
    img = img[bbox[1]:bbox[3], bbox[0]:bbox[2]]

    first = img
    return (first, second)